In [44]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
import gc
tqdm.pandas()
gc.collect()

C:\Users\ricky\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


497

In [45]:
%%time
df = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
df = df.loc[df.dt>=12,:] #取最近一年即可
print(df.shape,df.isnull().sum().sum())
df.head()

(19786604, 53) 1509444
Wall time: 1min 6s


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
13188928,12.0,10115966.0,15,1.0,9678.450445,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188938,12.0,10115966.0,18,1.0,9624.614033,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188944,12.0,10115966.0,25,1.0,2338.181448,1.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188946,12.0,10115966.0,26,2.0,16627.150110,2.0,0.0,0.0,0.0,1.00,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0
13188957,12.0,10115966.0,37,13.0,18485.113446,3.0,10.0,0.0,0.0,0.27,...,1.0,4.0,15.0,1.0,1.0,30.0,133000.939458,0.0,5.0,1.0


In [46]:
%%time
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
miss_table = missing_values_table(df)
miss_table

Your selected dataframe has 53 columns.
There are 9 columns that have missing values.
Wall time: 6.76 s


,Missing Values,% of Total Values
slam,1023527,5.2
gender_code,242749,1.2
age,242749,1.2
trdtp,365,0.0
educd,14,0.0
masts,10,0.0
naty,10,0.0
poscd,10,0.0
cuorg,10,0.0


In [47]:
mean_fill_col = ['slam'] # 數值型用中位數補值(避免離群值拉高平均值)
cat_fill_col = ['gender_code','age','trdtp','educd','masts','naty','poscd','cuorg'] # 類別型用一個-999補值
for i in tqdm(mean_fill_col):
    df[i] = df[i].fillna(df[i].median())
for i in tqdm(cat_fill_col):
    df[i] = df[i].fillna(-999)
miss_table = missing_values_table(df)
miss_table





100%|██████████| 1/1 [00:00<00:00,  2.70it/s]


















100%|██████████| 8/8 [00:01<00:00,  5.60it/s]


Your selected dataframe has 53 columns.
There are 0 columns that have missing values.


,Missing Values,% of Total Values


In [48]:
df.isnull().sum().sum()

0

In [49]:
chid = df.head(1)['chid'].values[0]
chid

10115966.0

In [50]:

官方指認欄位 = ['2','6','10','12','13','15','18','19','21','22','25','26','36','37','39','48']
def chid2answer(chid,method=None):
    if method in ['sum','mean','median']:
        a = df.loc[df.chid==chid,['shop_tag','txn_amt']].groupby('shop_tag').agg(method).sort_values(by='txn_amt',ascending=False)
    elif method in 'value_counts':
        a = df.loc[df.chid==chid,'shop_tag'].value_counts().to_frame()
    else:
        raise 'error'
    a['在指認欄位'] = False
    a.loc[list(set(a.index)&set(官方指認欄位)),'在指認欄位'] = True #有交集的部份做個記號
    answer = a[a['在指認欄位']==True].head(3)
    display(answer)
    return answer.index.tolist()

In [51]:
d = {}
for method in tqdm(['sum','mean','median','value_counts']):
    print(method)
    answer = chid2answer(chid,method=method)
    d[method] = answer
d

sum


,txn_amt,在指認欄位
shop_tag,,
2,232533.602882,True
37,220452.379787,True
18,199466.732709,True


mean


,txn_amt,在指認欄位
shop_tag,,
2,21139.418444,True
37,16957.875368,True
18,16622.227726,True


median


,txn_amt,在指認欄位
shop_tag,,
2,20273.605361,True
25,17534.700309,True
37,17415.509843,True


value_counts


,shop_tag,在指認欄位
37,13,True
18,12,True
2,11,True


100%|██████████| 4/4 [00:00<00:00, 30.03it/s]


{'sum': ['2', '37', '18'],
 'mean': ['2', '37', '18'],
 'median': ['2', '25', '37'],
 'value_counts': ['37', '18', '2']}

In [52]:
%%time
test = pd.read_feather('../data/需預測的顧客名單及提交檔案範例.feather')
test.head()

Wall time: 192 ms


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


In [53]:
test.loc[test.chid==chid]

,chid,top1,top2,top3
354911,10115966,12,15,48


In [54]:
method = 'value_counts'
test.loc[test.chid==chid,['top1','top2','top3']] = d[method]

In [55]:
test.loc[test.chid==chid]

,chid,top1,top2,top3
354911,10115966,37,18,2


In [56]:
test.to_csv(f'df_just_test_chid_{int(chid)}_method_{method}.csv',index=False)

In [57]:
df = pd.read_csv(f'df_just_test_chid_{int(chid)}_method_{method}.csv')
df

,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26
...,...,...,...,...
499995,10433239,18,48,25
499996,10235119,6,36,12
499997,10267595,21,26,15
499998,10160606,39,15,13


In [58]:
df2 = pd.read_csv('../data/需預測的顧客名單及提交檔案範例.csv')
df2

,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26
...,...,...,...,...
499995,10433239,18,48,25
499996,10235119,6,36,12
499997,10267595,21,26,15
499998,10160606,39,15,13


In [59]:
assert df.shape == df2.shape
assert df.isnull().sum().sum() == 0
assert (df.dtypes == df2.dtypes).all
print('test ok')

test ok
